In [1]:
from experimentlog import ExperimentLog
import extract
import pseudo
import pandas as pd
# log some JSON data
e = ExperimentLog(":memory:", ntp_sync=False)
# now register the user with the database
user = pseudo.get_pseudo()
e.create("USER", name=user, data={"age":30, "leftright":"right"})
e.cd("/Experiment/Condition1")
e.bind("USER",user)
e.log("mouse", data={"x":0, "y":0})
e.log("mouse", data={"x":0, "y":1})
e.log("mouse", data={"x":0, "y":2})
e.cd("/Experiment/Condition2")
e.bind("USER",user)
e.log("mouse", data={"x":0, "y":0})
e.log("mouse", data={"x":0, "y":1})
e.log("mouse", data={"x":0})
e.leave()
e.enter('Condition2')
e.log("mouse", data={"x":0, "y":0})
e.log("mouse", data={"x":0, "y":1})
e.log("mouse", data={"x":0}, valid=0)
e.enter('SubconditionA')
e.cd("/Experiment/Condition1")
e.bind("USER",user)
e.log("mouse", data={"x":0, "y":0})
e.log("mouse", data={"x":0, "y":1})
e.log("mouse", data={"x":0, "y":2})
e.cd("/Experiment/Condition2")
e.bind("USER",user)
e.log("mouse", data={"x":0, "y":0})
e.log("mouse", data={"x":0, "y":1})
e.log("mouse", data={"x":0})
e.leave()
e.enter('Condition2')
e.log("mouse", data={"x":0, "y":0})
e.log("mouse", data={"x":0, "y":1})
e.log("mouse", data={"x":0}, valid=0)
e.enter('SubconditionA')
e.end()


DEBUG:root:Opening database ':memory:'. Autocommit: 'None'
DEBUG:root:Creating tables.
DEBUG:root:Resuming from session 1 '/'
DEBUG:root:Run ID: [00000001]
DEBUG:root:Run config logged as '{}'
DEBUG:root:<Commit>
DEBUG:root:Registering 'KESOW-IBOSA' of type 'USER', with data [{"age": 30, "leftright": "right"}]
DEBUG:root:CD'ing to /Experiment/Condition1/
DEBUG:root:Parent session /
DEBUG:root:Entering session '/Experiment/'
DEBUG:root:New session ID [00000002]
DEBUG:root:<Commit>
DEBUG:root:Parent session /Experiment/
DEBUG:root:Entering session '/Experiment/Condition1/'
DEBUG:root:New session ID [00000003]
DEBUG:root:<Commit>
DEBUG:root:CD'd to /Experiment/Condition1/
DEBUG:root:Binding meta table USER:KESOW-IBOSA to /Experiment/Condition1/
02-16 17:05 [WARNI]  No stream mouse registered; creating a new blank entry
DEBUG:root:Registering 'mouse' of type 'STREAM', with data [null]
DEBUG:root:CD'ing to /Experiment/Condition2/
DEBUG:root:Leaving session '/Experiment/Condition1/'
DEBUG:ro

In [2]:
meta, bound_ix = extract.meta(e.cursor)
print extract.meta_dataframe(e.cursor)

{u'PATH':   bound  data description                                   name  type
0    []  None        None                           /Experiment/  None
1    []  None        None                /Experiment/Condition1/  None
2    []  None        None                /Experiment/Condition2/  None
3    []  None        None  /Experiment/Condition2/SubconditionA/  None, u'USER':           bound                                  data description  \
0  [3, 4, 7, 8]  {u'age': 30, u'leftright': u'right'}               

          name type  
0  KESOW-IBOSA       , u'STREAM':   bound  data description   name  type
0    []  None              mouse  AUTO, u'DATASET':   bound                 data description  name  type
0    []  {u'stage': u'init'}        None  None  None}


In [3]:
df = extract.dump_flat_dataframe(e.cursor)
print df['mouse'].describe()

         session             t   x          y      valid
count  18.000000  1.800000e+01  18  14.000000  18.000000
mean    6.000000  1.455642e+09   0   0.714286   0.888889
std     2.222876  8.188979e-03   0   0.726273   0.323381
min     3.000000  1.455642e+09   0   0.000000   0.000000
25%     4.000000  1.455642e+09   0   0.000000   1.000000
50%     6.000000  1.455642e+09   0   1.000000   1.000000
75%     8.000000  1.455642e+09   0   1.000000   1.000000
max     9.000000  1.455642e+09   0   2.000000   1.000000


In [4]:
df = extract.dump_dataframe(e.cursor)
print df


defaultdict(<type 'list'>, {(u'/Experiment/Condition2/SubconditionA/',): [{}, {}], (u'/Experiment/Condition1/',): [{u'mouse':    y  x  valid             t
0  0  0      1  1.455642e+09
1  1  0      1  1.455642e+09
2  2  0      1  1.455642e+09}, {u'mouse':    y  x  valid             t
0  0  0      1  1.455642e+09
1  1  0      1  1.455642e+09
2  2  0      1  1.455642e+09}], (u'/',): [{}], (u'/Experiment/',): [{}], (u'/Experiment/Condition2/',): [{u'mouse':     y  x  valid             t
0   0  0      1  1.455642e+09
1   1  0      1  1.455642e+09
2 NaN  0      1  1.455642e+09}, {u'mouse':     y  x  valid             t
0   0  0      1  1.455642e+09
1   1  0      1  1.455642e+09
2 NaN  0      0  1.455642e+09}, {u'mouse':     y  x  valid             t
0   0  0      1  1.455642e+09
1   1  0      1  1.455642e+09
2 NaN  0      1  1.455642e+09}, {u'mouse':     y  x  valid             t
0   0  0      1  1.455642e+09
1   1  0      1  1.455642e+09
2 NaN  0      0  1.455642e+09}]})


In [5]:
session_tree, path_tree = extract.map_children_sessions(e.cursor)
print "\n".join(["%-50s\tTotal logs:%d" % (k,len(v)) for k,v in path_tree.iteritems()])

/Experiment/                                      	Total logs:9
/Experiment/Condition1/                           	Total logs:2
/Experiment/Condition2/                           	Total logs:6
/Experiment/Condition2/SubconditionA/             	Total logs:2


In [7]:

print extract.dump_sessions_dataframe(e.cursor)

    complete description      end_time  json     last_time  log_count  \
id                                                                      
1        NaN        None           NaN  None           NaN          0   
2          0                       NaN  None           NaN          0   
3          1              1.455642e+09  None  1.455642e+09          3   
4          1              1.455642e+09  None  1.455642e+09          3   
5          1              1.455642e+09  None  1.455642e+09          3   
6          1              1.455642e+09  None           NaN          0   
7          1              1.455642e+09  None  1.455642e+09          3   
8          1              1.455642e+09  None  1.455642e+09          3   
9          0                       NaN  None  1.455642e+09          3   
10         0                       NaN  None           NaN          0   

             name  parent                                   path random_seed  \
id                                         